In [1]:
import { ArrayHelpers, DateTimeFormatWrapper } from "../utils/utils.ts";
import { getRecords } from "./transmission.ts";
import type { DateUploadSpeed, StatsRecord } from "./transmission.ts"

import { document } from "jsr:@ry/jupyter-helper";

import * as Plot from "npm:@observablehq/plot";

const RAW_RECORDS = await getRecords("transmission.csv");

const LOCALE = new Intl.Locale("fr-FR");

const uploadSpeedFormat = new Intl.NumberFormat(LOCALE, {
  notation: "compact",
  style: "unit",
  unit: "byte-per-second",
});

console.table(RAW_RECORDS);


┌───────┬──────────────────────────┬─────────────────┬────────────┬───────────────┬──────────────┬─────────────────┬────────────────────┬───────────────┬────────────────────┬──────────────┬─────────────┐
│ (idx) │ date                     │ downloadedBytes │ filesAdded │ secondsActive │ sessionCount │ uploadedBytes   │ activeTorrentCount │ downloadSpeed │ pausedTorrentCount │ torrentCount │ uploadSpeed │
├───────┼──────────────────────────┼─────────────────┼────────────┼───────────────┼──────────────┼─────────────────┼────────────────────┼───────────────┼────────────────────┼──────────────┼─────────────┤
│     0 │ 2025-07-16T21:11:27.759Z │   3925170815043 │     237665 │      57124256 │          708 │ 171586887710452 │                183 │             0 │                255 │          438 │     1000000 │
│     1 │ 2025-07-16T21:11:40.640Z │   3925170815043 │     237665 │      57124269 │          708 │ 171586900710452 │                183 │             0 │                255 │          

In [7]:
const byHours = new ArrayHelpers(RAW_RECORDS)
  .map((v) => {
    const date = new Date(v.date);
    date.setMinutes(0, 0, 0);
    return {
      ...v,
      date,
    };
  })
  .groupBy((v) => v.date.getTime())
  .values()
  .map<DateUploadSpeed>((values) => {
    const dest: DateUploadSpeed = Object.create(null);
    dest.date = values.getFirst().date;
    dest.uploadSpeed = Math.round(values.average(e => e.uploadSpeed));
    return dest;
  })
  .toArray();
const byHoursFormatter = new DateTimeFormatWrapper(LOCALE, {
  weekday: "short",
  day: "numeric",
  hour: "numeric",
  hour12: false,
});

Plot.plot({
  document,
  title: "Vitesse d’envoi par heure",
  x: {
    tickFormat: byHoursFormatter.tickFormat((m) =>
      `${m.get("weekday")} ${m.get("day")}\n${m.get("hour")}h`
    ),
  },
  marks: [
    Plot.barY(byHours, {
      x: "date",
      y: "uploadSpeed",
      margin: 50,
    }),
    Plot.line(byHours, {
      x: "date",
      y: "uploadSpeed",
      stroke: "red",
      curve: "natural",
      margin: 50,
      marker: "dot",
    }),
    Plot.text(byHours, {
      x: "date",
      y: "uploadSpeed",
      text: (d) => uploadSpeedFormat.format(d.uploadSpeed),
      dy: -10,
    }),
  ],
  // margin: moy.length * 3,
  width: byHours.length * 50,
  figure: true,
});


Vitesse d’envoi par heure 0 1,000,000 2,000,000 3,000,000 4,000,000 5,000,000 6,000,000 7,000,000 8,000,000 9,000,000 10,000,000 11,000,000 ↑ uploadSpeed mer. 16 23h jeu. 17 00h jeu. 17 01h jeu. 17 02h jeu. 17 03h jeu. 17 04h jeu. 17 05h jeu. 17 06h jeu. 17 07h jeu. 17 08h jeu. 17 09h jeu. 17 10h jeu. 17 11h jeu. 17 12h jeu. 17 13h jeu. 17 14h jeu. 17 15h jeu. 17 16h jeu. 17 17h jeu. 17 18h jeu. 17 19h jeu. 17 20h jeu. 17 21h jeu. 17 22h jeu. 17 23h ven. 18 00h ven. 18 01h ven. 18 02h ven. 18 03h ven. 18 04h ven. 18 05h ven. 18 06h ven. 18 07h ven. 18 08h ven. 18 09h ven. 18 10h ven. 18 11h ven. 18 12h ven. 18 13h ven. 18 14h ven. 18 15h ven. 18 16h ven. 18 17h ven. 18 18h ven. 18 19h ven. 18 20h ven. 18 21h ven. 18 22h ven. 18 23h sam. 19 00h sam. 19 01h sam. 19 02h sam. 19 03h sam. 19 04h sam. 19 05h sam. 19 06h sam. 19 07h sam. 19 08h sam. 19 09h sam. 19 10h sam. 19 11h sam. 19 12h sam. 19 13h sam. 19 14h sam. 19 15h sam. 19 16h sam. 19 17h sam. 19 18h sam. 19 19h sam. 19 20h sam. 19 21h sam. 19 22h sam. 19 23h dim. 20 00h dim. 20 01h dim. 20 02h dim. 20 03h dim. 20 04h dim. 20 05h dim. 20 06h dim. 20 07h dim. 20 08h dim. 20 09h dim. 20 10h dim. 20 11h dim. 20 12h dim. 20 13h dim. 20 14h dim. 20 15h dim. 20 16h dim. 20 17h dim. 20 18h dim. 20 19h dim. 20 20h dim. 20 21h dim. 20 22h dim. 20 23h lun. 21 00h lun. 21 01h lun. 21 02h lun. 21 03h lun. 21 04h lun. 21 05h lun. 21 06h lun. 21 07h lun. 21 08h lun. 21 09h lun. 21 10h lun. 21 11h lun. 21 12h lun. 21 13h lun. 21 14h lun. 21 15h lun. 21 16h lun. 21 17h lun. 21 18h lun. 21 19h lun. 21 20h lun. 21 21h lun. 21 22h lun. 21 23h mar. 22 00h mar. 22 01h mar. 22 02h mar. 22 03h mar. 22 04h mar. 22 05h mar. 22 06h mar. 22 07h mar. 22 08h mar. 22 09h mar. 22 10h mar. 22 11h mar. 22 12h mar. 22 13h mar. 22 14h mar. 22 15h mar. 22 16h mar. 22 17h mar. 22 18h mar. 22 19h mar. 22 20h mar. 22 21h mar. 22 22h mar. 22 23h mer. 23 00h mer. 23 01h mer. 23 02h mer. 23 03h mer. 23 04h mer. 23 05h mer. 23 06h mer. 23 07h mer. 23 08h mer. 23 09h mer. 23 10h mer. 23 11h mer. 23 12h mer. 23 13h mer. 23 14h mer. 23 15h mer. 23 16h mer. 23 17h mer. 23 18h mer. 23 19h mer. 23 20h mer. 23 21h mer. 23 22h mer. 23 23h jeu. 24 00h jeu. 24 01h jeu. 24 02h jeu. 24 03h jeu. 24 04h jeu. 24 05h jeu. 24 06h jeu. 24 07h jeu. 24 08h jeu. 24 09h jeu. 24 10h jeu. 24 11h jeu. 24 12h jeu. 24 13h jeu. 24 14h jeu. 24 15h jeu. 24 16h jeu. 24 17h jeu. 24 18h jeu. 24 19h jeu. 24 20h jeu. 24 21h jeu. 24 22h jeu. 24 23h ven. 25 00h ven. 25 01h ven. 25 02h ven. 25 03h ven. 25 04h ven. 25 05h ven. 25 06h ven. 25 07h ven. 25 08h ven. 25 09h ven. 25 10h ven. 25 11h ven. 25 12h ven. 25 13h ven. 25 14h ven. 25 15h ven. 25 16h ven. 25 17h ven. 25 18h ven. 25 20h ven. 25 21h ven. 25 22h ven. 25 23h sam. 26 00h sam. 26 01h sam. 26 02h sam. 26 03h sam. 26 04h sam. 26 05h sam. 26 06h sam. 26 07h sam. 26 08h sam. 26 09h sam. 26 10h sam. 26 11h sam. 26 12h sam. 26 13h sam. 26 14h sam. 26 15h sam. 26 16h sam. 26 17h sam. 26 18h <path d="M54.955,323.217C71.473,319.136,87.991,315.055,104.509,317.343C121.027,319.631,137.544,328.288,154.062,326.13C170.58,323.971,187.098,310.996,203.615,302.663C220.133,294.331,236.651,290.642,253.169,290.539C269.687,290.436,286.204,293.919,302.722,303.158C319.24,312.397,335.758,327.391,352.276,329.5C368.793,331.609,385.311,320.833,401.829,314.697C418.347,308.561,434.865,307.066,451.382,315.711C467.9,324.355,484.418,343.139,500.936,350C517.454,356.861,533.971,351.798,550.489,350C567.007,348.202,583.525,349.668,600.043,345.883C616.56,342.098,633.078,333.062,649.596,332.202C666.114,331.342,682.632,338.657,699.149,340.485C715.667,342.313,732.185,338.653,748.703,319.372C765.22,300.092,781.738,265.19,798.256,266.662C814.774,268.135,831.292,305.981,847.809,315.469C864.327,324.957,880.845,306.086,897.363,301.876C913.881,297.666,930.398,308.116,946.916,317.267C963.434,326.418,979.952,334.269,996.47,307.937C1012.987,281.605,1029.505,221.09,1046.023,198.809C1062.541,176.529,1079.059,192.483,1095.576,212.356C1112.094,

In [8]:
const moyByHours = new ArrayHelpers(RAW_RECORDS)
  .map((v) => {
    const date = new Date(0);
    date.setHours(v.date.getHours());
    return {
      ...v,
      date,
    };
  })
  .groupBy(({ date }) => date.getTime())
  .values()
  .map<DateUploadSpeed>((values) => {
    const dest: DateUploadSpeed = Object.create(null);
    dest.date = values.getFirst().date;
    dest.uploadSpeed = Math.round(values.average(v => v.uploadSpeed));
    return dest;
  })
  .toArray()
  .toSorted((a, b) => a.date.getTime() - b.date.getTime());

const moyByHoursFormatter = new DateTimeFormatWrapper(LOCALE, {
  hour: "2-digit",
  hour12: false,
});

Plot.plot({
  title: "Moyenne de vitesse d’envoi par heure",
  document,
  x: {
    tickFormat: moyByHoursFormatter.tickFormat((m) => `${m.get("hour")}h`),
  },
  marks: [
    Plot.barY(moyByHours, {
      x: "date",
      y: "uploadSpeed",
    }),
    Plot.line(moyByHours, {
      x: "date",
      y: "uploadSpeed",
      stroke: "red",
      curve: "natural",
      marker: "dot",
    }),
    Plot.text(moyByHours, {
      x: "date",
      y: "uploadSpeed",
      text: (d) => uploadSpeedFormat.format(d.uploadSpeed),
      dy: -10,
    }),
  ],
  margin: 50,
  // margin: moy.length * 3,
  width: moyByHours.length * 50,
  figure: true,
});


Moyenne de vitesse d’envoi par heure 0 200,000 400,000 600,000 800,000 1,000,000 1,200,000 1,400,000 1,600,000 1,800,000 2,000,000 2,200,000 2,400,000 ↑ uploadSpeed 00h 01h 02h 03h 04h 05h 06h 07h 08h 09h 10h 11h 12h 13h 14h 15h 16h 17h 18h 19h 20h 21h 22h 23h <path d="M62,143.701C77,192.668,92,241.636,107,207.626C122,173.616,137,56.628,152,50.922C167,45.216,182,150.792,197,161.169C212,171.546,227,86.724,242,104.545C257,122.367,272,242.832,287,265.012C302,287.192,317,211.086,332,169.214C347,127.342,362,119.703,377,173.258C392,226.814,407,341.564,422,387.971C437,434.377,452,412.44,467,395.256C482,378.073,497,365.643,512,340.317C527,314.99,542,276.767,557,256.036C572,235.304,587,232.066,602,235.655C617,239.245,632,249.662,647,241.418C662,233.175,677,206.27,692,201.826C707,197.382,722,215.398,737,216.466C752,217.533,767,201.651,782,200.102C797,198.553,812,211.336,827,199.467C842,187.597,857,151.074,872,124.655C887,98.237,902,81.923,917,70.198C932,58.474,947,51.34,962,52.595C977,53.85,992,63.494,1007,90.969C1022,118.445,1037,163.752,1052,159.896C1067,156.039,1082,103.02,1097,50" marker-end="url(#plot-marker-6)" marker-mid="url(#plot-marker-6)" marker-start="url(#plot-marker-6)" /> 1,9 M o/s 1,4 M o/s 2,6 M o/s 1,8 M o/s 2,2 M o/s 1 M o/s 1,7 M o/s 1,7 M o/s 89 k o/s 35 k o/s 442 k o/s 1,1 M o/s 1,2 M o/s 1,2 M o/s 1,5 M o/s 1,4 M o/s 1,5 M o/s 1,5 M o/s 2 M o/s 2,4 M o/s 2,6 M o/s 2,3 M o/s 1,8 M o/s 2,6 M o/s ⚠️ 1 warning. Please check the console.